# Convert FITS files to a format for Bruce

In [1]:
# imports
from astropy.io import fits

# Load

In [2]:
cpd = fits.open('CPD-56_F.fits')

In [3]:
cpd.info()

Filename: CPD-56_F.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      84   (15558,)   float32   


## Header

In [6]:
cpd[0].header

SIMPLE  =                    T /                                                
BITPIX  =                  -32 /                                                
NAXIS   =                    1 /                                                
EXTEND  =                    T /Extensions may be present                       
BSCALE  =           1.00000000         / real=bzero+bscale*value                
BUNIT   = 'DU/PIXEL'                                                            
ORIGIN  = 'LCO/OCIW'                                                            
OBSERVER= 'P. Massey'                  / observer name                          
TELESCOP= 'Baade_Mag_1'                / telescope                              
SITENAME= 'LCO'                                                                 
SITEALT =                 2405         / meters                                 
SITELAT =            -29.01423         / latitude                               
SITELONG=            -70.692

### Log-linear

In [8]:
cdelt1 = cpd[0].header['CDELT1']
crpix1 = cpd[0].header['CRPIX1']
crval1 = cpd[0].header['CRVAL1']